<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week7_Day4_XP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ex1

In [2]:
# !pip install -q faiss-cpu==1.7.4
# !pip install -q chromadb==0.3.21
! pip install -qU chromadb
! pip install -q numpy<2
! mkdir cache
!apt install libomp-dev
!python -m pip install --upgrade faiss-cpu
import numpy as np
import pandas as pd
import faiss
import json
from sentence_transformers import SentenceTransformer, InputExample
import chromadb
from chromadb.config import Settings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s eta 

In [3]:
!mkdir cache

mkdir: cannot create directory ‘cache’: File exists


In [5]:
import pandas as pd
path =  "/content/labelled_newscatcher_dataset.csv"
pdf = pd.read_csv(path, sep=';')
pdf.head()

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en


In [6]:
pdf["id"] = pdf.index.astype(str)
print(pdf[["id"]].head())

  id
0  0
1  1
2  2
3  3
4  4


In [7]:
display(pdf)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


In [8]:
from sentence_transformers import InputExample

In [9]:
pdf_subset = pdf.head(1000).copy()
pdf_subset

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
995,TECHNOLOGY,https://www.androidcentral.com/mate-40-will-be...,androidcentral.com,2020-08-07 17:12:33,The Mate 40 will be the last Huawei phone with...,en,995
996,SCIENCE,https://www.cnn.com/2020/08/17/africa/stone-ag...,cnn.com,2020-08-17 17:10:00,"Early humans knew how to make comfy, pest-free...",en,996
997,HEALTH,https://www.tenterfieldstar.com.au/story/68776...,tenterfieldstar.com.au,2020-08-13 03:26:06,Regional Vic set for virus testing blitz,en,997
998,HEALTH,https://news.sky.com/story/coronavirus-trials-...,news.sky.com,2020-08-13 13:22:58,Coronavirus: Trials of second contact-tracing ...,en,998


In [11]:
def example_create_fn(doc1: pd.Series) -> InputExample:
    # Helper function that converts a single row into an InputExample.
    # It uses the row's 'id' as guid and 'title' as the text for embedding.
    guid = doc1["id"]
    text = str(doc1["title"])  # Assure-toi que la colonne "title" existe
    return InputExample(guid=guid, texts=[text], label=0.0)

In [12]:
faiss_train_examples = pdf_subset.apply(example_create_fn, axis=1).tolist()

In [14]:
faiss_train_examples[:10]

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# Extract the Titles and Convert to a List of Strings
titles_list = pdf_subset["title"].tolist()

In [16]:
# Supposons que pdf_subset contient la colonne "title" avec les titres
titles = pdf_subset["title"].astype(str).tolist()  # convertir en liste de chaînes

# Générer les embeddings
faiss_title_embedding = model.encode(titles, convert_to_numpy=True)

In [17]:
len(faiss_title_embedding), len(faiss_title_embedding[0])

(1000, 384)

In [18]:
import numpy as np
import faiss

print("NumPy version:", np.__version__)
print("FAISS imported successfully!")


NumPy version: 2.0.2
FAISS imported successfully!


In [19]:
# Ensure the subset and embeddings exist
pdf_subset = pdf.head(1000).copy()
pdf_subset["id"] = pdf_subset.index.astype(str)

# Generate embeddings for the 'title' column
titles = pdf_subset["title"].astype(str).tolist()
model = SentenceTransformer('all-MiniLM-L6-v2')
faiss_title_embedding = model.encode(titles, convert_to_numpy=True)

# Attach embeddings to the DataFrame
pdf_subset["embedding"] = list(faiss_title_embedding)

# ✅ Prepare for indexing
pdf_to_index = pdf_subset[["id", "title", "embedding"]].copy()
id_index = pdf_to_index["id"].tolist()



In [20]:
content_encoded_normalized = faiss_title_embedding.copy()  # copy embeddings to avoid overwriting original
faiss.normalize_L2(content_encoded_normalized)  # normalize each vector to unit length (L2 norm)

In [21]:
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))
index_content.add_with_ids(content_encoded_normalized, id_index)

In [22]:
def search_content(query, pdf_to_index, k=3):
    # Encode the query string into an embedding vector and convert to float32 numpy array
    query_vector = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_vector)  # Normalize the query vector in-place

    # Perform the search: index_content is your FAISS index from earlier
    distances, ids = index_content.search(query_vector, k)  # ids shape: (1, k)

    # Flatten ids and distances arrays for easier processing
    ids = ids[0]
    similarities = distances[0]

    # Retrieve matching articles from pdf_to_index by filtering by 'id'
    # Ensure 'id' column type matches the ids returned by FAISS
    matched_articles = pdf_to_index[pdf_to_index["id"].astype(int).isin(ids)].copy()

    # Map similarities to articles by matching ids
    # Create a dict for quick similarity lookup
    sim_dict = dict(zip(ids, similarities))

    # Add similarity scores to DataFrame in the correct order
    matched_articles["similarities"] = matched_articles["id"].astype(int).map(sim_dict)

    # Sort results by similarity descending
    results = matched_articles.sort_values(by="similarities", ascending=False)

    return results


In [23]:
display(search_content("animal", pdf_to_index, k=5))

,id,title,embedding,similarities
176,176,Random: You Can Pick Up and Pet Cats in Assass...,"[0.040390525, -0.007339595, -0.038180403, 0.04...",0.391902
975,975,Researchers explore social behavior of animals...,"[0.019667476, 0.0033410194, 0.019479454, 0.065...",0.376784
99,99,Ghostwire: Tokyo confirms dog petting,"[-0.04976743, -0.034099158, 0.08398495, 0.0354...",0.344058
928,928,Just Let This Lizard Be a Dinosaur,"[-0.06954565, 0.048526313, -0.0095300665, -0.0...",0.317387
762,762,'Secret' life of sharks: Study reveals their s...,"[-0.05458369, 0.00788134, 0.034392443, 0.08967...",0.295497


In [26]:
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.Client()
collection_name = "my_news"

# If a collection with the same name exists, delete it to avoid conflicts
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)

print(f"Creating collection: '{collection_name}'")
collection = chroma_client.create_collection(name=collection_name)

# Display the DataFrame subset (for reference)
display(pdf_subset)

collection.add(
    documents=pdf_subset["title"][:100].tolist(),
    metadatas=[{"topic": topic} for topic in pdf_subset["topic"][:100].tolist()],
    ids=pdf_subset["id"][:100].tolist()
)


Creating collection: 'my_news'


,topic,link,domain,published_date,title,lang,id,embedding
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0,"[-0.112705484, 0.040765435, 0.02181417, 0.0551..."
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1,"[-0.021871628, -0.03349994, 0.073218025, 0.039..."
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2,"[0.016083736, 0.002794495, -0.015044215, 0.060..."
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3,"[0.003022199, 0.048076212, 0.07132513, 0.03649..."
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4,"[0.06794903, 0.025916306, 0.13602017, -0.01073..."
...,...,...,...,...,...,...,...,...
995,TECHNOLOGY,https://www.androidcentral.com/mate-40-will-be...,androidcentral.com,2020-08-07 17:12:33,The Mate 40 will be the last Huawei phone with...,en,995,"[-0.08776905, 0.01518162, 0.052281417, -0.0326..."
996,SCIENCE,https://www.cnn.com/2020/08/17/africa/stone-ag...,cnn.com,2020-08-17 17:10:00,"Early humans knew how to make comfy, pest-free...",en,996,"[0.046402894, 0.07256383, 0.03314629, 0.081965..."
997,HEALTH,https://www.tenterfieldstar.com.au/story/68776...,tenterfieldstar.com.au,2020-08-13 03:26:06,Regional Vic set for virus testing blitz,en,997,"[0.015069275, 0.04583016, -0.061145045, -0.066..."
998,HEALTH,https://news.sky.com/story/coronavirus-trials-...,news.sky.com,2020-08-13 13:22:58,Coronavirus: Trials of second contact-tracing ...,en,998,"[-0.07082235, 0.006438215, 0.008093198, -0.026..."


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 65.5MiB/s]


In [28]:
results = collection.query(
    query_texts=["space"],
    n_results=10,
    include=["documents", "metadatas"]  # Removed "ids"
)

import json
print(json.dumps(results, indent=4))


{
    "ids": [
        [
            "72",
            "7",
            "30",
            "26",
            "23",
            "76",
            "69",
            "40",
            "47",
            "75"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Beck teams up with NASA and AI for 'Hyperspace' visual album experience",
            "Orbital space tourism set for rebirth in 2021",
            "NASA drops \"insensitive\" nicknames for cosmic objects",
            "\u2018It came alive:\u2019 NASA astronauts describe experiencing splashdown in SpaceX Dragon",
            "Hubble Uses Moon As \u201cMirror\u201d to Study Earth\u2019s Atmosphere \u2013 Proxy in Search of Potentially Habitable Planets Around Other Stars",
            "Australia's small yet crucial part in the mission to find life on Mars",
            "NASA Astronauts in SpaceX Capsule Splashdown in Gulf Of Mexico",
            "SpaceX's Starship spacecraft saw 150 meters high",
          

Exercise 5: Question Answering with Hugging Face Model

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"  # Specify the model ID, e.g., 'gpt2'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the causal language model
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [30]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Maximum number of tokens to generate.
    device_map="auto",   # Automatically uses available GPU/CPU resources.
)


Device set to use cpu


In [31]:
# Define the user's question
question = "What's the latest news on space development?"

# Concatenate the retrieved documents into a single context string
context = " ".join(results["documents"][0])  # assuming results["documents"][0] is a list of texts

# Build the prompt template combining context and question
prompt_template = f"Relevant context: {context}\n\nThe user's question: {question}"


In [32]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relevant context: Beck teams up with NASA and AI for 'Hyperspace' visual album experience Orbital space tourism set for rebirth in 2021 NASA drops "insensitive" nicknames for cosmic objects ‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon Hubble Uses Moon As “Mirror” to Study Earth’s Atmosphere – Proxy in Search of Potentially Habitable Planets Around Other Stars Australia's small yet crucial part in the mission to find life on Mars NASA Astronauts in SpaceX Capsule Splashdown in Gulf Of Mexico SpaceX's Starship spacecraft saw 150 meters high NASA’s InSight lander shows what’s beneath Mars’ surface Alien base on Mercury: ET hunters claim to find huge UFO

The user's question: What's the latest news on space development?

Space exploration is a long, complicated, and expensive endeavor. It's hard to understand the world without looking at the vast amount of resources that go into it. This piece is about the "what" of the NASA space program, and what NAS